# Applied Data Analysis Project
**Team**: ToeStewBrr - Alexander Sternfeld, Marguerite Thery, Antoine Bonnet, Hugo Bordereaux

**Dataset**: CMU Movie Summary Corpus


## CoreNLP Analysis

[**CoreNLP**](https://nlp.stanford.edu/software/) is an incredible natural language processing toolkit created at Stanford University. CoreNLP is applied through a **pipeline** of sequential analysis steps called annotators. The full list of available annotators is available [here](https://stanfordnlp.github.io/CoreNLP/annotators.html). 

As described by its creators: 

*"CoreNLP is your one stop shop for natural language processing in Java! CoreNLP enables users to derive linguistic annotations for text, including token and sentence boundaries, parts of speech, named entities, numeric and time values, dependency and constituency parses, coreference, sentiment, quote attributions, and relations. CoreNLP currently supports 8 languages: Arabic, Chinese, English, French, German, Hungarian, Italian, and Spanish."* 

You can create your own pipeline to extract the desired information. You can try it out for yourself in this [online shell](https://corenlp.run).

### Loading data
We first load data files and download the pre-processed dataframes. 

In [47]:
from zipfile import ZipFile

from load_data import *
from coreNLP_analysis import *

download_data(coreNLP=False)
plot_df = load_plot_df()
movie_df = load_movie_df()
char_df = load_char_df()
names_df = load_names_df()
cluster_df = load_cluster_df()

### 1. Exploring pre-processed CoreNLP data

The authors of the Movie CMU dataset used CoreNLP to parse each plot summary to extract various linguistic insights. In this section, we explore how much information we can gather from these pre-processed files. 

We will use *Harry Potter*'s character throughout this section.

#### 1.1. Character data

For any character, we first extract related information from the provided name clusters and character metadata.

In [48]:
# Given character, extract all pre-processed dataframe data
char_name = 'Harry Potter'
movie_ids = list(char_df[char_df['Character name'] == char_name]['Wikipedia ID'])
char_ids = names_df.loc[char_name].values[0]
trope = cluster_df.loc[cluster_df['Character name'] == char_name]

# If no trope is found, set it to None
if trope.empty:
    trope = None

print('Movies with character', char_name, ':')
print('\tMovie IDs:', movie_ids)
print('\tCharacter IDs:', char_ids)
print('\tTrope:', trope)

movie_id = movie_ids[3] 
movie_name = movie_df.loc[movie_df['Wikipedia ID'] == movie_id]['Name'].iloc[0]
print('Selecting as example: \n\tMovie ID:', movie_id, '\n\tMovie title:', movie_name)


Movies with character Harry Potter :
	Movie IDs: [858575, 667372, 670407, 31941988, 9834441, 667368, 667371, 667361, 667361]
	Character IDs: ['/m/0jz6jt', '/m/02tbbh6', '/m/0jz6mq', '/m/0jz6hs', '/m/02tbf6n', '/m/0jz6b0', '/m/0jz6dz', '/m/09lybcb']
	Trope: None
Selecting as example: 
	Movie ID: 31941988 
	Movie title: Harry Potter and the Deathly Hallows – Part 2


#### 1.2. Extracting sentences

We now extract information from the CoreNLP plot summary analysis. The authors of the dataset stored the analysis output of each movie into a `.xml` file. Each file has a tree structure detailing each word of each sentence as well as the parsed sentence in tree form. 

We now extract all parsed sentences from the `.xml` files. 

A **parsed sentence** is a syntactic analysis tree, where each word is a leaf tagged by its lexical function (e.g. *VBZ* for verbs or *DT* for determinants). Semantic interactions between different words are also indicated within the structure of the tree. 

In [49]:
# Extract the tree of xml file and all parsed sentences
tree = get_tree(movie_id)
sentences = get_parsed_sentences(tree)

# Picking the fifth sentence as example
parsed_str = sentences[5]
print(parsed_str)
print_tree(parsed_str)

(ROOT (S (PP (IN In) (NP (NP (NNP Bellatrix) (POS 's)) (NN vault))) (, ,) (NP (NNP Harry)) (VP (VBZ discovers) (SBAR (S (NP (DT the) (NNP Horcrux)) (VP (VBZ is) (NP (NP (NNP Helga) (NNP Hufflepuff) (POS 's)) (NN cup)))))) (. .))) 
                                                ROOT                                                 
                                                 |                                                    
                                                 S                                                   
                _________________________________|_________________________________________________   
               |             |    |                               VP                               | 
               |             |    |        _______________________|____                            |  
               |             |    |       |                           SBAR                         | 
               |             |    |       |         

#### 1.3. Extracting characters

We also want to extract all character names directly from the xml file. Note that we aggregate consecutive words tagged as NNP (noun, proper, singular) as the same character name (this assumes that plot summaries never contain two distinct names side by side without delimiting punctuation). This is a reasonable assumption since list of names are almost always separated by commas. 

In [50]:
characters = get_characters(tree)
print(characters[:20])

['Voldemort', 'Albus Dumbledore', 'Severus Snape', 'Dobby', 'Harry Potter', 'Ron', 'Hermione', 'Griphook', 'Harry', 'Ollivander', 'Ollivander', 'Draco Malfoy', 'Malfoy', 'Harry', 'Harry', 'Helga Hufflepuff', 'Griphook', 'Harry', 'Voldemort', 'Griphook']


Notice that some characters are sometimes mentioned by their full name, and sometimes by a partial name (e.g. Harry Potter is most often mentioned as simply Harry). To get a more precise idea of how many times each character is mentioned, we wish to denote each character by their full name, i.e. the longest version of their name that appears in the plot summary. 

*NOTE*: The dataset has the character metadata of only a third of the movies, so we need to extract full names from the plot summary itself and not the provided dataframes. 

To optimize full name lookup, for each plot summary we construct a dictionary which stores as key every partial name mentioned, and as corresponding values the full name of each character.  

In [51]:
char_name = 'Albus'
full_name = get_full_name(char_name, characters)
print('Example: the full name of "{}" is "{}".'.format(char_name,full_name))
print('Full name dictionary:', full_name_dict(characters))

Example: the full name of "Albus" is "Albus Dumbledore".
Full name dictionary: {'Voldemort': 'Voldemort', 'Albus Dumbledore': 'Albus Dumbledore', 'Severus Snape': 'Severus Snape', 'Dobby': 'Dobby', 'Harry Potter': 'Harry Potter', 'Ron': 'Ron', 'Hermione': 'Hermione Weasley', 'Griphook': 'Griphook', 'Harry': 'Harry Potter', 'Ollivander': 'Ollivander', 'Draco Malfoy': 'Draco Malfoy', 'Malfoy': 'Draco Malfoy', 'Helga Hufflepuff': 'Helga Hufflepuff', 'Rowena Ravenclaw': 'Rowena Ravenclaw', 'Hogsmeade': 'Hogsmeade', 'Aberforth Dumbledore': 'Aberforth Dumbledore', 'Ariana': 'Ariana', 'Neville Longbottom': 'Neville Longbottom', 'Snape': 'Severus Snape', 'Minerva McGonagall': 'Minerva McGonagall', 'Luna Lovegood': 'Luna Lovegood', 'Helena Ravenclaw': 'Helena Ravenclaw', 'Gregory Goyle': 'Gregory Goyle', 'Blaise Zabini': 'Blaise Zabini', 'Nagini': 'Nagini', 'Fred': 'Fred', 'Lily': 'Lily', 'James': 'James', 'Dumbledore': 'Albus Dumbledore', 'Neville': 'Neville Longbottom', 'Molly Weasley': 'Moll

We can now extract the most mentioned characters in any plot summary, in descending order of frequency. We can then see that Harry Potter is indeed the main character of the movie, as he is mentioned 26 times, more than any other character in the summary.  

In [52]:
char_mentions = most_mentioned(movie_id)
print(char_mentions)

[('Harry Potter', 26), ('Voldemort', 21), ('Severus Snape', 11), ('Ron', 6), ('Hermione Weasley', 6), ('Albus Dumbledore', 5), ('Griphook', 3), ('Draco Malfoy', 3), ('Neville Longbottom', 3), ('Nagini', 3), ('Ollivander', 2), ('Lily', 2), ('Dobby', 1), ('Helga Hufflepuff', 1), ('Rowena Ravenclaw', 1), ('Hogsmeade', 1), ('Aberforth Dumbledore', 1), ('Ariana', 1), ('Minerva McGonagall', 1), ('Luna Lovegood', 1), ('Helena Ravenclaw', 1), ('Gregory Goyle', 1), ('Blaise Zabini', 1), ('Fred', 1), ('James', 1), ('Molly Weasley', 1), ('Ginny Potter', 1)]


 #### 1.4. Extracting interactions

We are also interested in character interactions. We can use the number of common mentions of two characters in the same sentence as a proxy for the number of interactions. For any movie, we find the number of common mentions (i.e. interactions) for each pair of characters. 

In [53]:
char_pairs = character_pairs(movie_id, plot_df)
print(char_pairs[:10])

[(('Hermione Weasley', 'Ron'), 4), (('Harry Potter', 'Voldemort'), 4), (('Albus Dumbledore', 'Voldemort'), 3), (('Albus Dumbledore', 'Severus Snape'), 2), (('Harry Potter', 'Hermione Weasley'), 2), (('Harry Potter', 'Ron'), 2), (('Nagini', 'Voldemort'), 2), (('Harry Potter', 'Lily'), 2), (('Albus Dumbledore', 'Harry Potter'), 2), (('Severus Snape', 'Voldemort'), 1)]


In [54]:
main_interaction = character_pairs(movie_id, plot_df)[0][0]
print('Main interaction in the movie:', main_interaction)

Main interaction in the movie: ('Hermione Weasley', 'Ron')


#### 1.5. Extracting characters and interactions of all movies

We will now use the above code to obtain the main character and main interaction for every plot summary. 

*NOTE*: This code takes a while to run, so you can load the analysis from a pre-processed file instead.  

In [55]:
# NOTE: If we've already run this code, we can load the dataframe from a file
pairs_df = pd.read_csv('Data/MovieSummaries/plot_characters.csv', sep='\t', index_col=0)
pairs_df

,Wikipedia ID,Summary,Main character,Number of mentions,Main interaction,Number of interactions
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Shlykov,1.0,"('Lyosha', 'Shlykov')",1.0
1,31186339,The nation of Panem consists of a wealthy Capi...,Katniss,18.0,"('Katniss', 'Peeta Mellark')",2.0
2,20663735,Poovalli Induchoodan is sentenced for six yea...,Maranchery Karunakara Menon,9.0,"('Manapally Madhavan Nambiar', 'judge Menon')",1.0
3,2231378,"The Lemon Drop Kid , a New York City swindler,...",Charley,18.0,NaN,NaN
4,595909,Seventh-day Adventist Church pastor Michael Ch...,Lindy,7.0,"('Azaria', 'Lindy')",1.0
...,...,...,...,...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho...",Reema,1.0,"('Muslim', 'Reema')",1.0
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look...",Leo Andreyev,7.0,NaN,NaN
42300,35102018,American Luthier focuses on Randy Parsons’ tra...,Randy Parsons,4.0,NaN,NaN
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se...",Abdur Rehman Khan,9.0,"('Abdur Rehman Khan', 'Amina')",1.0


In [ ]:
# Otherwise: get main character and number of mentions for each movie and store it into a file (takes a while to run) 
pairs_df = plot_df.copy(deep=True)
pairs_df['Main character'] = pairs_df['Wikipedia ID'].apply(most_mentioned)
pairs_df['Number of mentions'] = pairs_df['Main character'].apply(lambda x: np.nan if x is None else x[0][1])
pairs_df['Main character'] = pairs_df['Main character'].apply(lambda x: np.nan if x is None else x[0][0])

# Get main pairs of characters for each movie and number of interactions 
pairs_df['Main interaction'] = pairs_df['Wikipedia ID'].apply(lambda x: character_pairs(x, plot_df))
pairs_df['Number of interactions'] = pairs_df['Main interaction'].apply(lambda x: np.nan if x is None else x[0][1])
pairs_df['Main interaction'] = pairs_df['Main interaction'].apply(lambda x: np.nan if x is None else x[0][0])

# Store data into csv file
pairs_df.to_csv('Data/MovieSummaries/plot_characters.csv', sep='\t')^
pairs_df

In conclusion, the coreNLP files provided with the datasets are useful to extract the characters mentioned. 

 However, our goal is to extract love relationships as well as the persona of characters in love. Using common mentions as a proxy for love relationships is a vulgar approximation and so we must run our own NLP analysis on the plot summaries to extract useful information. 

### 2. Custom CoreNLP Analysis

We now use a **custom CoreNLP pipeline** to analyze the plot summaries. Due to the weakness of our available computing power, we will only analyze romantic comedy movies for now. 

Our custom pipeline consists of the following annotators: 

1. [Tokenization (tokenize)](https://stanfordnlp.github.io/CoreNLP/tokenize.html): Turns the whole text into tokens. 

2. [Parts Of Speech (POS)](https://stanfordnlp.github.io/CoreNLP/pos.html): Tags each token with part of speech labels (e.g. determinants, verbs and nouns). 

3. [Lemmatization (lemma)](https://stanfordnlp.github.io/CoreNLP/lemma.html): Reduces each word to its lemma (e.g. *was* becomes *be*). 

4. [Named Entity Recognition (NER)](https://stanfordnlp.github.io/CoreNLP/ner.html): Identifies named entities from the text, including characters, locations and organizations. 

5. [Constituency parsing (parse)](https://stanfordnlp.github.io/CoreNLP/parse.html): Performs a syntactic analysis of each sentence in the form of a tree. 

6. [Coreference resolution (coref)](https://stanfordnlp.github.io/CoreNLP/coref.html): Aggregates mentions of the same entities in a text (e.g. when 'Harry' and 'he' refer to the same person). 

7. [Dependency parsing (depparse)](https://stanfordnlp.github.io/CoreNLP/depparse.html): Syntactic dependency parser. 

8. [Natural Logic (natlog)](https://stanfordnlp.github.io/CoreNLP/natlog.html): Identifies quantifier scope and token polarity. Required as preliminary for OpenIE. 

9. [Open Information Extraction (OpenIE)](https://stanfordnlp.github.io/CoreNLP/openie.html): Identifies relation between words as triples *(subject, relation, object of relation)*. We use this to extract relationships between characters, as well as character traits. 

10. [Knowledge Base Population (KBP)](https://stanfordnlp.github.io/CoreNLP/kbp.html): Identifies meaningful relation triples. 


#### 2.1. Running our pipeline

We now run our own CoreNLP analysis on the plot summaries. This allows us to extract love relationships from the plot summaries much more accurately. 

**Goal**: Run our custom CoreNLP pipeline. 

**Recommendation**: Be careful about memory storage (takes a lot of memory to run!)

**Prerequisite**: [java](https://www.java.com). 

**Installation steps**:
1. Download the CoreNLP toolkit [here](https://stanfordnlp.github.io/CoreNLP/download.html).

2. Change directory (`cd`) into the downloaded `stanford-corenlp` directory. 

3. Data preparation: Extract plot summaries for romantic comedies into `.txt` files. Create a filelist containing the name of all the files which need to be processed using the following command in your terminal: 

<center>find RomancePlots/*.txt > filelist.txt</center>
        
4. Run the custom CoreNLP pipeline via your terminal using the following command:

<center>java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,pos,lemma,ner,parse,coref,depparse,natlog,openie,kbp -coref.md.type RULE -filelist filelist.txt -outputDirectory RomancePlotsOutputs/ -outputFormat xml</center>

We take a look at the romantic comedy plot summaries we passed through our pipeline. 

In [58]:
# Get a dataframe with romantic movies and their corresponding plots
romance_genres = ['Romantic comedy'] 
# For later use: romance_genres = ['Romantic comedy', 'Romance Film', 'Romantic drama', 'Romantic fantasy', 'Romantic thriller']
rom_com_plots = get_plots(romance_genres, movie_df, plot_df)
rom_com_plots

,Wikipedia ID,Summary
0,6631279,"Adam, a San Francisco-based artist who works a..."
1,21926710,Jimmy ([[Hiroshi Watanabe loves dinosaurs and...
2,26067101,Perry is an English chemist working for a pain...
3,12053509,"Randy Bodek is a rebellious college slacker, ..."
4,7028314,"Sir Philip Ashlow , his neglected wife, Lady A..."
...,...,...
2675,1001692,"Leslie Steele , a guest at a costume party is ..."
2676,25655938,Kate Tosconi is a journalist in her early 20s ...
2677,11823946,Viola and Sebastian are young twins and ente...
2678,30553937,A daytime soap opera star has to deal with his...


Extract all romantic comedies plots into separate txt files to be able to run them through the new coreNLP pipeline

In [ ]:
# for index, row in rom_com_plots.iterrows():
#     with open("Data/MovieSummaries/RomancePlots/{}.txt".format(row['Wikipedia ID']), 'w') as f:
#         if type(row['Summary']) == str:
#             f.write(row['Summary'])
#             f.close()

In [ ]:
# Unzip 
with ZipFile('Romance_Data/RomancePlotsOutputs.zip', 'r') as zipObj:
   # Extract all the romance plots xml files
   zipObj.extractall('')

For each xml file representing a romantic movie, we extract the kbp title relationship. 
TODO: Rerun corenlp on the files 43849.txt.xml and 1282593.txt.xml which cannot be parsed as trees. Update the zip. 

In [59]:
# To be moved to python file once done
# Create a list of tuples containing (movie_id, subject, object) for each kbp triples with title relationship
def get_relation_df(DIR, relation_type): 
    title = []
    for filename in os.listdir(DIR):
        # Manually deleted files: 43849.txt.xml and 1282593.txt.xml because could not be parsed
        if filename != ".DS_Store" and filename != "43849.txt.xml" and filename != "1282593.txt.xml":
            movie_id = filename[:-8]
            title.append(get_relation(movie_id, relation_type))
    title_df = pd.DataFrame([item for sublist in title for item in sublist], columns=['Wikipedia ID', 'Subject', 'Title'])
    title_df = title_df.groupby(['Wikipedia ID','Subject'])['Title'].apply(', '.join).reset_index()  
    return title_df   

# List of relevant relationships to chose from: 
- per_age
- per_alternate_names
- per_cause_of_death
- per_children
- per_cities_of_residence
- per_city_of_birth
- per_city_of_death
- per_countries_of_residence
- per_country_of_birth
- per_country_of_death
- per_date_of_birth
- per_date_of_death
- per_employee_of
- per_member_of
- per_origin
- per_other_family
- per_parents
- per_religion
- per_schools_attended
- per_siblings
- per_spouse
- per_stateorprovince_of_birth
- per_stateorprovince_of_death
- per_stateorprovinces_of_residence
- per_title

In [60]:
title_df = get_relation_df(DIR = 'RomancePlotsOutputs/', relation_type = 'per:title')
title_df

,Wikipedia ID,Subject,Title
0,1002717,Delia Darrow,assistant
1,1002717,Fergie,Inspector
2,1002717,Gerda Caswell,assistant
3,1002717,Gloria Mundy,"librarian, manager"
4,1002717,Rachel Roberts,assistant
...,...,...,...
1904,4339865,Max de Mirecourt,writer
1905,4339865,Tam Tam,Princess
1906,4340003,Danny Miller,producer
1907,4340003,Max Corkle,messenger


RomancePlotsOutputs has 1491 readable files. 

In [61]:
love_df = get_relation_df(DIR = 'RomancePlotsOutputs/', relation_type = 'per:spouse')
love_df

,Wikipedia ID,Subject,Title
0,1001692,Steele,Lady Claire
1,1006385,Lindsay,Rose
2,10095235,Diogo Álvares,"Diogo Álvares, Paraguaçu, Diogo Álvares"
3,10095235,Paraguaçu,Diogo Álvares
4,10190365,Paro,Shambunath Mehta
...,...,...,...
518,431333,Toula,Toula
519,4339865,Lucie,Max de Mirecourt
520,4339865,Max de Mirecourt,Lucie
521,4343386,Kathy,Peter


In [ ]:
death_df = get_relation_df(DIR = 'RomancePlotsOutputs/', relation_type = 'per:cause_of_death')
death_df